In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# pip install google-adk

# Setup Google API Key

In [3]:
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print ( "Google API Key setup complete")
except exception as e:
    print ( f" Google API Key Exception: {e} ")

Google API Key setup complete


# Importing Google ADK and Python Module

In [4]:
import re
import socket
from datetime import datetime
import json
import requests
import subprocess
import time
import uuid

from typing import Any, Dict

from google.adk.agents import Agent, LlmAgent
from google.adk.apps.app import App, EventsCompactionConfig
from google.adk.models.google_llm import Gemini
from google.adk.sessions import DatabaseSessionService
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
# from google.adk.tools.tool_context import ToolContext, load_memory, preload_memory
from google.genai import types
from google.adk.tools import google_search
from google.adk.agents import LlmAgent
from google.adk.agents.remote_a2a_agent import (
    RemoteA2aAgent,
    AGENT_CARD_WELL_KNOWN_PATH,
)

from google.adk.a2a.utils.agent_to_a2a import to_a2a
from google.adk.models.google_llm import Gemini
# from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
# from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [5]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

# User Method for login information

In [6]:


sessionId= ""
email= ""
fullName=""
phone=""
class userInfo:
    email:str
    fullName : str
    phone :str
    sessionId: str

user=userInfo()


def is_valid_email(email: str) -> bool:
    # Very simple email validation
    pattern = r"^[^@\s]+@[^@\s]+\.[^@\s]+$"
    return re.match(pattern, email) is not None

def is_valid_phone(phone: str) -> bool:
    # Accept digits, spaces, +, -, (, )
    cleaned = re.sub(r"[^\d]", "", phone)
    return 7 <= len(cleaned) <= 15

def getSessionName(email:str) -> str:
    userStr=email.split("@")[0] +"_"+ socket.gethostname()
    now = datetime.now()
    # Format as string, e.g. 2025-11-30 09:33:00
    timestamp = now.strftime("%Y-%m-%dT%H%M%S")
    sessName = userStr +"_"+timestamp
    print(sessName)
    return sessName

    

def collect_user_info():
    cond=True
    while cond:
        email = input("Enter your email: ").strip()
        if not is_valid_email(email):
            print("Invalid email format. Please try again.")
            continue
        user.email=email
        user.sessionId =getSessionName(email)
        full_name = input("Enter your full name: ").strip()
        if len(full_name.split()) < 2:
            print("Please enter your full name (first and last).")
            continue
        user.fullName =full_name
        phone = input("Enter your phone number: ").strip()
        if not is_valid_phone(phone):
            print("Invalid phone number. Please try again.")
            continue
        user.phone =phone
        print("\nThanks! Your details have been recorded.")
        cond=False
        # return {
        #     "email": email,
        #     "full_name": full_name,
        #     "phone": phone,
        #     "session": sessionId
        # }

# def chatbot_loop(user_info):
#     print(f"\nHi {user_info['full_name']}! You can start chatting now.")
#     print("Type 'exit' to end the chat.\n")

#     while True:
#         user_msg = input("You: ").strip()
#         if user_msg.lower() in ("exit", "quit"):
#             print("Bot: Thanks for chatting. Goodbye!")
#             break

#         # Very simple bot logic (placeholder)
#         if "hello" in user_msg.lower():
#             print("Bot: Hello! How can I help you today?")
#         elif "name" in user_msg.lower():
#             print(f"Bot: You told me your name is {user_info['full_name']}.")
#         elif "phone" in user_msg.lower():
#             print(f"Bot: You registered with phone number {user_info['phone']}.")
#         elif "email" in user_msg.lower():
#             print(f"Bot: Your email is {user_info['email']}.")
#         else:
#             print("Bot: I am a simple demo bot. Ask me about your name, email, or phone.")

def main():
    print("Welcome to the demo chatbot!")
    collect_user_info()
    # fullName=user_info['full_name']
    # phone=user_info['phone']
    # email=user_info['email']
    # sessionId=user_info['session']
    
    print(user.sessionId)
    # chatbot_loop(user)

if __name__ == "__main__":
    main()

# fullName=user_info['full_name']
# phone=user_info['phone']
# email=user_info['email']
# sessionId=user_info['session']


Welcome to the demo chatbot!


Enter your email:  satish.orre@outlook.com


satish.orre_ca1134161095_2025-12-01T185620


Enter your full name:  Satish Orre
Enter your phone number:  1234232



Thanks! Your details have been recorded.
satish.orre_ca1134161095_2025-12-01T185620


# Section 2: Expose the Product Catalog Agent via A2A

In [7]:
# Define Product Tool for getting production information

def getProductInfo(productName: str) -> dict:
    """
      This method return all matches products from API
    """
    url= f'https://dummyjson.com/products/search?q={productName}'
    print(url)
    resp = requests.get(url, timeout=10)

    # Raise if HTTP status is not 2xx
    resp.raise_for_status()
    products= []
    # print(products, len(products))
    # if len(products) == 0:
    #     return {
    #         'id': 0
    #         ,'title': f"Sorry, I don't have information for {productName}. Available products: 0"
    #     }
    data=resp.json()
    counter=0
    count=0
    # print(type(data), len(data) )
    for item in data['products']:
        if productName.lower() in item.get('title').lower():
            products.append(item)
            count += 1
        counter+= 1
    if len(products) == 0:
        return {
            'id': 0
            ,'title': f"Sorry, I don't have information for {productName}. Available products: 0"
        }
    # print(counter, count)
    # Parse and return JSON body
    return products
    


# print(getProductInfo('Iphone 6'))

In [8]:
# Define Product Tool for getting production information
import requests

def getAllProductCategories() -> dict:
    """
      This method return all product Categories from API
    """
    url= f'https://dummyjson.com/products/categories'
    print(url)
    resp = requests.get(url, timeout=10)

    # Raise if HTTP status is not 2xx
    resp.raise_for_status()
    products= []
    print(resp)
    data=resp.json()
    return data
    


# print( getAllProductCategories())

In [9]:
# Define Product Tool for getting production information
import requests

def getAllProductsbyCategory(category:str ) -> dict:
    """
      This method return all product Categories from API
    """
    url= f'https://dummyjson.com/products/category/{category}'
    print(url)
    resp = requests.get(url, timeout=10)

    # Raise if HTTP status is not 2xx
    resp.raise_for_status()
    products= []
    print(resp)
    products=resp.json()['products']
    return products
    
def getInventorybyCategory(category: list[str] |str = None) -> dict:
    products = []
    if isinstance(category, list):
         for ls in category:
             temp=getAllProductsbyCategory(ls)
             for item in temp:
                 selectElements= ["id",  "title", "category", "stock"]
                 new_dict = {k: item[k] for k in selectElements if k in item}
                 products.append(new_dict)
    elif isinstance(category, str):
        original=getAllProductsbyCategory(category)
        # print(len(original))
        for item in original:
            # print(type(item), item)
            selectElements= ["id",  "title", "category", "stock"]
            new_dict = {k: item[k] for k in selectElements if k in item}
            # print(new_dict)
            products.append(new_dict)
            
    return products

# print( getAllProductsbyCategory('beauty'))
# print(getInventorybyCategory('beauty' ))

In [10]:
# Create the Product Catalog Agent
# This agent specializes in providing product information from the vendor's catalog
product_catalog_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="product_catalog_agent",
    description="External vendor's product catalog agent that provides product information and availability.",
    instruction="""
    You are a product catalog specialist from an external vendor.
    When asked about products, use the get_product_info tool to fetch data from the catalog.
    Provide clear, accurate product information including price, availability, and specs.
    If asked about multiple products, look up each one.
    Provide clear, accurate product categories
    Be professional and helpful.
    """,
    tools=[getProductInfo],  # Register the product lookup tool
)

print("✅ Product Catalog Agent created successfully!")
print("   Model: gemini-2.5-flash-lite")
print("   Tool: getProductInfo()")
print("   Ready to be exposed via A2A...")

✅ Product Catalog Agent created successfully!
   Model: gemini-2.5-flash-lite
   Tool: getProductInfo()
   Ready to be exposed via A2A...


In [11]:
# Convert the product catalog agent to an A2A-compatible application
# This creates a FastAPI/Starlette app that:
#   1. Serves the agent at the A2A protocol endpoints
#   2. Provides an auto-generated agent card
#   3. Handles A2A communication protocol
product_catalog_a2a_app = to_a2a(
    product_catalog_agent, port=8001  # Port where this agent will be served
)

print("✅ Product Catalog Agent is now A2A-compatible!")
print("   Agent will be served at: http://localhost:8001")
print("   Agent card will be at: http://localhost:8001/.well-known/agent-card.json")
print("   Ready to start the server...")

✅ Product Catalog Agent is now A2A-compatible!
   Agent will be served at: http://localhost:8001
   Agent card will be at: http://localhost:8001/.well-known/agent-card.json
   Ready to start the server...


/tmp/ipykernel_47/3453858527.py:6: UserWarning: [EXPERIMENTAL] to_a2a: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  product_catalog_a2a_app = to_a2a(
/usr/local/lib/python3.11/dist-packages/google/adk/a2a/utils/agent_to_a2a.py:137: UserWarning: [EXPERIMENTAL] A2aAgentExecutor: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  agent_executor = A2aAgentExecutor(
/usr/local/lib/python3.11/dist-packages/google/adk/a2a/executor/a2a_agent_executor.py:101:

# Section 3: Start the Product Catalog Agent Server

In [12]:
# First, let's save the product catalog agent to a file that uvicorn can import
product_catalog_agent_code = '''
import os
import requests
from google.adk.agents import LlmAgent
from google.adk.a2a.utils.agent_to_a2a import to_a2a
from google.adk.models.google_llm import Gemini
from google.genai import types

retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

def getProductInfo(productName: str) -> dict:
    """
      This method return all matches products from API
    """
    url= f'https://dummyjson.com/products/search?q={productName}'
    print(url)
    resp = requests.get(url, timeout=10)

    # Raise if HTTP status is not 2xx
    resp.raise_for_status()
    products= []
    # print(products, len(products))
    # if len(products) == 0:
    #     return {
    #         'id': 0
    #         ,'title': f"Sorry, I don't have information for {productName}. Available products: 0"
    #     }
    data=resp.json()
    counter=0
    count=0
    # print(type(data), len(data) )
    for item in data['products']:
        if productName.lower() in item.get('title').lower():
            products.append(item)
            count += 1
        counter+= 1
    if len(products) == 0:
        return {
            'id': 0
            ,'title': f"Sorry, I don't have information for {productName}. Available products: 0"
        }
    # print(counter, count)
    # Parse and return JSON body
    return products

    
product_catalog_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="product_catalog_agent",
    description="External vendor's product catalog agent that provides product information and availability.",
    instruction="""
    You are a product catalog specialist from an external vendor.
    When asked about products, use the getProductInfo tool to fetch data from the catalog.
    Provide clear, accurate product information including price, availability, and specs.
    If asked about multiple products, look up each one. 
    Be professional and helpful.
    """,
    tools=[getProductInfo],
)

# Create the A2A app
app = to_a2a(product_catalog_agent, port=8001)
'''

# Write the product catalog agent to a temporary file
with open("/tmp/product_catalog_server.py", "w") as f:
    f.write(product_catalog_agent_code)

print("📝 Product Catalog agent code saved to /tmp/product_catalog_server.py")

# Start uvicorn server in background
# Note: We redirect output to avoid cluttering the notebook
server_process = subprocess.Popen(
    [
        "uvicorn",
        "product_catalog_server:app",  # Module:app format
        "--host",
        "localhost",
        "--port",
        "8001",
    ],
    cwd="/tmp",  # Run from /tmp where the file is
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    env={**os.environ},  # Pass environment variables (including GOOGLE_API_KEY)
)

print("🚀 Starting Product Catalog Agent server...")
print("   Waiting for server to be ready...")

# Wait for server to start (poll until it responds)
max_attempts = 30
for attempt in range(max_attempts):
    try:
        response = requests.get(
            "http://localhost:8001/.well-known/agent-card.json", timeout=1
        )
        if response.status_code == 200:
            print(f"\n✅ Product Catalog Agent server is running!")
            print(f"   Server URL: http://localhost:8001")
            print(f"   Agent card: http://localhost:8001/.well-known/agent-card.json")
            break
    except requests.exceptions.RequestException:
        time.sleep(5)
        print(".", end="", flush=True)
else:
    print("\n⚠️  Server may not be ready yet. Check manually if needed.")

# Store the process so we can stop it later
globals()["product_catalog_server_process"] = server_process

📝 Product Catalog agent code saved to /tmp/product_catalog_server.py
🚀 Starting Product Catalog Agent server...
   Waiting for server to be ready...
.....
✅ Product Catalog Agent server is running!
   Server URL: http://localhost:8001
   Agent card: http://localhost:8001/.well-known/agent-card.json


In [13]:
# Fetch the agent card from the running server
try:
    response = requests.get(
        "http://localhost:8001/.well-known/agent-card.json", timeout=5
    )

    if response.status_code == 200:
        agent_card = response.json()
        print("📋 Product Catalog Agent Card:")
        print(json.dumps(agent_card, indent=2))

        print("\n✨ Key Information:")
        print(f"   Name: {agent_card.get('name')}")
        print(f"   Description: {agent_card.get('description')}")
        print(f"   URL: {agent_card.get('url')}")
        print(f"   Skills: {len(agent_card.get('skills', []))} capabilities exposed")
    else:
        print(f"❌ Failed to fetch agent card: {response.status_code}")

except requests.exceptions.RequestException as e:
    print(f"❌ Error fetching agent card: {e}")
    print("   Make sure the Product Catalog Agent server is running (previous cell)")

📋 Product Catalog Agent Card:
{
  "capabilities": {},
  "defaultInputModes": [
    "text/plain"
  ],
  "defaultOutputModes": [
    "text/plain"
  ],
  "description": "External vendor's product catalog agent that provides product information and availability.",
  "name": "product_catalog_agent",
  "preferredTransport": "JSONRPC",
  "protocolVersion": "0.3.0",
  "skills": [
    {
      "description": "External vendor's product catalog agent that provides product information and availability. \n    I am a product catalog specialist from an external vendor.\n    When asked about products, use the getProductInfo tool to fetch data from the catalog.\n    Provide clear, accurate product information including price, availability, and specs.\n    If asked about multiple products, look up each one. \n    Be professional and helpful.\n    ",
      "id": "product_catalog_agent",
      "name": "model",
      "tags": [
        "llm"
      ]
    },
    {
      "description": "This method return all m

 # Local Sub-agent created

In [14]:
print(AGENT_CARD_WELL_KNOWN_PATH)

# Create a RemoteA2aAgent that connects to our Product Catalog Agent
# This acts as a client-side proxy - the Customer Support Agent can use it like a local agent
remote_product_catalog_agent = RemoteA2aAgent(
    name="product_catalog_agent",
    description="Remote product catalog agent from external vendor that provides product information.",
    # Point to the agent card URL - this is where the A2A protocol metadata lives
    agent_card=f"http://localhost:8001{AGENT_CARD_WELL_KNOWN_PATH}",
)

print("✅ Remote Product Catalog Agent proxy created!")
print(f"   Connected to: http://localhost:8001")
print(f"   Agent card: http://localhost:8001{AGENT_CARD_WELL_KNOWN_PATH}")
print("   The Customer Support Agent can now use this like a local sub-agent!")

/.well-known/agent-card.json
✅ Remote Product Catalog Agent proxy created!
   Connected to: http://localhost:8001
   Agent card: http://localhost:8001/.well-known/agent-card.json
   The Customer Support Agent can now use this like a local sub-agent!


/tmp/ipykernel_47/3233624445.py:5: UserWarning: [EXPERIMENTAL] RemoteA2aAgent: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  remote_product_catalog_agent = RemoteA2aAgent(


In [15]:
# #Re-define our app with Events Compaction enabled
# research_app_compacting = App(
#     name="research_app_compacting",
#     root_agent=chatbot_agent,
#     # This is the new part!
#     events_compaction_config=EventsCompactionConfig(
#         compaction_interval=3,  # Trigger compaction every 3 invocations
#         overlap_size=1,  # Keep 1 previous turn for context
#     ),
# )

# db_url = "sqlite:///my_agent_data.db"  # Local SQLite file
# session_service = DatabaseSessionService(db_url=db_url)

# # Create a new runner for our upgraded app
# research_runner_compacting = Runner(
#     app=research_app_compacting, session_service=session_service
# )


# print("✅ Research App upgraded with Events Compaction!")

# Customer Support Agent Setup 

In [16]:
# Now create the Customer Support Agent that uses the remote Product Catalog Agent
customer_support_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="customer_support_agent",
    description="A customer support assistant that helps customers with product inquiries and information.",
    instruction="""
    You are a friendly and professional customer support agent.
    
    When customers ask about products:
    1. Use the product_catalog_agent sub-agent to look up product information
    2. Provide clear answers about pricing, availability, and specifications
    3. If a product is out of stock, mention the expected availability
    4. Be helpful and professional!
        
    Always get product information from the product_catalog_agent before answering customer questions
    """,
    sub_agents=[remote_product_catalog_agent ],  # Add the remote agent as a sub-agent!
)

print("✅ Customer Support Agent created!")
print("   Model: gemini-2.5-flash-lite")
print("   Sub-agents: 1 (remote Product Catalog Agent via A2A)")
print("   Ready to help customers!")

#Re-define our app with Events Compaction enabled
customer_support_app = App(
    name="CustomerSupportApp",
    root_agent=customer_support_agent,
    # This is the new part!
    events_compaction_config=EventsCompactionConfig(
        compaction_interval=3,  # Trigger compaction every 3 invocations
        overlap_size=1,  # Keep 1 previous turn for context
    ),
)


✅ Customer Support Agent created!
   Model: gemini-2.5-flash-lite
   Sub-agents: 1 (remote Product Catalog Agent via A2A)
   Ready to help customers!


/tmp/ipykernel_47/3339318700.py:30: UserWarning: [EXPERIMENTAL] EventsCompactionConfig: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  events_compaction_config=EventsCompactionConfig(


# Section 5 Test A2A Communication

In [18]:
async def test_a2a_communication(user_query: str):
    """
    Test the A2A communication between Customer Support Agent and Product Catalog Agent.

    This function:
    1. Creates a new session for this conversation
    2. Sends the query to the Customer Support Agent for products, for 
    3. Support Agent communicates with Product Catalog Agent via A2A
    4. Displays the response
   
    Always get product information from the product_catalog_agent before answering customer questions
    
    Args:
        user_query: The question to ask the Customer Support Agent
    """
    # Setup session management (required by ADK)
    # session_service = InMemorySessionService()
    db_url = "sqlite:///my_agent_data.db"  # Local SQLite file
#     session_service = DatabaseSessionService(db_url=db_url)

#     # Session identifiers
#     app_name = "support_app"
#     # user_id = "demo_user"
#     user_id = user.email
#     # Use unique session ID for each test to avoid conflicts
#     session_id = f"demo_session_{uuid.uuid4().hex[:8]}"
#     # session_id = user.sessionId

#     # CRITICAL: Create session BEFORE running agent (synchronous, not async!)
#     # This pattern matches the deployment notebook exactly
#     session = await session_service.create_session(app_name=app_name, user_id=user_id, session_id=session_id)
        
#     # print(session, type(session))

#     # Create runner for the Customer Support Agent
#     # The runner manages the agent execution and session state
#     runner = Runner(
#         agent=customer_support_app, app_name=app_name, session_service=session_service
#     )
#     # runner = Runner(
#     #     app=customer_support_app,  session_service=session_service
#     # )
#     # Create the user message
#     # This follows the same pattern as the deployment notebook
#     test_content = types.Content(parts=[types.Part(text=user_query)])

#     # Display query
#     print(f"\n👤 Customer: {user_query}")
#     print(f"\n🎧 Support Agent response:")
#     print("-" * 60)

#     # Run the agent asynchronously (handles streaming responses and A2A communication)
#     async for event in runner.run_async(
#         user_id=user_id, session_id=session_id, new_message=test_content
#     ):
#         # Print final response only (skip intermediate events)
#         if event.is_final_response() and event.content:
#             for part in event.content.parts:
#                 if hasattr(part, "text"):
#                     print(part.text)

#     print("-" * 60)


# # Run the test
# print("🧪 Testing A2A Communication...\n")
# await test_a2a_communication("Can you tell me about the iPhone 15 Pro? Is it in stock?")

INFO:google_adk.google.adk.sessions.database_session_service:Local timezone: Etc/UTC


🧪 Testing A2A Communication...


👤 Customer: Can you tell me about the iPhone 15 Pro? Is it in stock?

🎧 Support Agent response:
------------------------------------------------------------


ValidationError: 1 validation error for InvocationContext
agent
  Input should be a valid dictionary or instance of BaseAgent [type=model_type, input_value=App(name='CustomerSupport...esumability_config=None), input_type=App]
    For further information visit https://errors.pydantic.dev/2.12/v/model_type

In [25]:
async def test_a2a_communication(user_query: str):
    """
    Test the A2A communication between Customer Support Agent and Product Catalog Agent.

    This function:
    1. Creates a new session for this conversation
    2. Sends the query to the Customer Support Agent
    3. Support Agent communicates with Product Catalog Agent via A2A
    4. Displays the response

    Args:
        user_query: The question to ask the Customer Support Agent
    """
    # Setup session management (required by ADK)
    # session_service = InMemorySessionService()
    db_url = "sqlite:///my_agent_data.db"  # Local SQLite file
    session_service = DatabaseSessionService(db_url=db_url)    

   # Session identifiers
    app_name = "support_app"
    # user_id = "demo_user"
    user_id = user.email
    # Use unique session ID for each test to avoid conflicts
    session_id = f"demo_session_{uuid.uuid4().hex[:8]}"
    # session_id = user.sessionId

    # CRITICAL: Create session BEFORE running agent (synchronous, not async!)
    # This pattern matches the deployment notebook exactly
    session = await session_service.create_session(
        app_name=app_name, user_id=user_id, session_id=session_id
    )

    # Create runner for the Customer Support Agent
    # The runner manages the agent execution and session state
    runner = Runner(
        agent=customer_support_agent, app_name=app_name, session_service=session_service
    )

    # Create the user message
    # This follows the same pattern as the deployment notebook
    test_content = types.Content(parts=[types.Part(text=user_query)])

    # Display query
    print(f"\n👤 Customer: {user_query}")
    print(f"\n🎧 Support Agent response:")
    print("-" * 60)

    # Run the agent asynchronously (handles streaming responses and A2A communication)
    async for event in runner.run_async(
        user_id=user_id, session_id=session_id, new_message=test_content
    ):
        # Print final response only (skip intermediate events)
        if event.is_final_response() and event.content:
            for part in event.content.parts:
                if hasattr(part, "text"):
                    print(part.text)

    print("-" * 60)


# Run the test
print("🧪 Testing A2A Communication...\n")
await test_a2a_communication("Can you tell me about the iPhone 15 Pro? Is it in stock?")

INFO:google_adk.google.adk.sessions.database_session_service:Local timezone: Etc/UTC
INFO:google_adk.google.adk.models.google_llm:Sending out request, model: gemini-2.5-flash-lite, backend: GoogleLLMVariant.GEMINI_API, stream: False


🧪 Testing A2A Communication...


👤 Customer: Can you tell me about the iPhone 15 Pro? Is it in stock?

🎧 Support Agent response:
------------------------------------------------------------


INFO:google_adk.google.adk.models.google_llm:Response received from the model.


I am sorry, but I don't have any information regarding the iPhone 15 Pro in my product catalog.
------------------------------------------------------------


In [26]:
print("🧪 Testing A2A Communication...\n")
await test_a2a_communication("Can you tell me about the iPhone 5s? Is it in stock?")

INFO:google_adk.google.adk.sessions.database_session_service:Local timezone: Etc/UTC
INFO:google_adk.google.adk.models.google_llm:Sending out request, model: gemini-2.5-flash-lite, backend: GoogleLLMVariant.GEMINI_API, stream: False


🧪 Testing A2A Communication...


👤 Customer: Can you tell me about the iPhone 5s? Is it in stock?

🎧 Support Agent response:
------------------------------------------------------------


INFO:google_adk.google.adk.models.google_llm:Response received from the model.


The iPhone 5s is currently in stock. It is a classic smartphone with a compact design. It is priced at $199.99. Would you like to know anything else about it?
------------------------------------------------------------


In [ ]:
# Define helper functions that will be reused throughout the notebook

from IPython.core.display import display, HTML
from jupyter_server.serverapp import list_running_servers


# Gets the proxied URL in the Kaggle Notebooks environment
def get_adk_proxy_url():
    PROXY_HOST = "https://kkb-production.jupyter-proxy.kaggle.net"
    ADK_PORT = "8000"

    servers = list(list_running_servers())
    if not servers:
        raise Exception("No running Jupyter servers found.")

    baseURL = servers[0]["base_url"]

    try:
        path_parts = baseURL.split("/")
        kernel = path_parts[2]
        token = path_parts[3]
    except IndexError:
        raise Exception(f"Could not parse kernel/token from base URL: {baseURL}")

    url_prefix = f"/k/{kernel}/{token}/proxy/proxy/{ADK_PORT}"
    url = f"{PROXY_HOST}{url_prefix}"

    styled_html = f"""
    <div style="padding: 15px; border: 2px solid #f0ad4e; border-radius: 8px; background-color: #fef9f0; margin: 20px 0;">
        <div style="font-family: sans-serif; margin-bottom: 12px; color: #333; font-size: 1.1em;">
            <strong>⚠️ IMPORTANT: Action Required</strong>
        </div>
        <div style="font-family: sans-serif; margin-bottom: 15px; color: #333; line-height: 1.5;">
            The ADK web UI is <strong>not running yet</strong>. You must start it in the next cell.
            <ol style="margin-top: 10px; padding-left: 20px;">
                <li style="margin-bottom: 5px;"><strong>Run the next cell</strong> (the one with <code>!adk web ...</code>) to start the ADK web UI.</li>
                <li style="margin-bottom: 5px;">Wait for that cell to show it is "Running" (it will not "complete").</li>
                <li>Once it's running, <strong>return to this button</strong> and click it to open the UI.</li>
            </ol>
            <em style="font-size: 0.9em; color: #555;">(If you click the button before running the next cell, you will get a 500 error.)</em>
        </div>
        <a href='{url}' target='_blank' style="
            display: inline-block; background-color: #1a73e8; color: white; padding: 10px 20px;
            text-decoration: none; border-radius: 25px; font-family: sans-serif; font-weight: 500;
            box-shadow: 0 2px 5px rgba(0,0,0,0.2); transition: all 0.2s ease;">
            Open ADK Web UI (after running cell below) ↗
        </a>
    </div>
    """

    display(HTML(styled_html))

    return url_prefix


print("✅ Helper functions defined.")

# Review User Logs from database

In [27]:
import sqlite3

def check_data_in_db():
    with sqlite3.connect("my_agent_data.db") as connection:
        cursor = connection.cursor()
        result = cursor.execute(
            "select app_name, session_id, author, content from events"
        )
        print([_[0] for _ in result.description])
        for each in result.fetchall():
            print(each)


check_data_in_db()

['app_name', 'session_id', 'author', 'content']
('support_app', 'demo_session_c8a7c53e', 'user', '{"parts": [{"text": "Can you tell me about the iPhone 15 Pro? Is it in stock?"}], "role": "user"}')
('support_app', 'demo_session_c8a7c53e', 'customer_support_agent', '{"parts": [{"function_call": {"id": "adk-ad7feb50-2941-4c7f-86f9-371c470aae07", "args": {"agent_name": "product_catalog_agent"}, "name": "transfer_to_agent"}}], "role": "model"}')
('support_app', 'demo_session_c8a7c53e', 'customer_support_agent', '{"parts": [{"function_response": {"id": "adk-ad7feb50-2941-4c7f-86f9-371c470aae07", "name": "transfer_to_agent", "response": {"result": null}}}], "role": "user"}')
('support_app', 'demo_session_c8a7c53e', 'product_catalog_agent', '{"parts": [{"text": "I am sorry, but I could not find any information for the iPhone 15 Pro in our catalog."}], "role": "model"}')
('support_app', 'demo_session_aaa8671c', 'user', '{"parts": [{"text": "Can you tell me about the iPhone 5s? Is it in stock?"

In [ ]:
# Define helper functions that will be reused throughout the notebook
async def run_session(
    runner_instance: Runner,
    user_queries: list[str] | str = None,
    session_name: str = "default",
):
    print(f"\n ### Session: {session_name}")

    # Get app name from the Runner
    app_name = runner_instance.app_name

    # Attempt to create a new session or retrieve an existing one
    try:
        session = await session_service.create_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )
    except:
        session = await session_service.get_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )

    # Process queries if provided
    if user_queries:
        # Convert single query to list for uniform processing
        if type(user_queries) == str:
            user_queries = [user_queries]

        # Process each query in the list sequentially
        for query in user_queries:
            print(f"\nUser > {query}")

            # Convert the query string to the ADK Content format
            query = types.Content(role="user", parts=[types.Part(text=query)])

            # Stream the agent's response asynchronously
            async for event in runner_instance.run_async(
                user_id=USER_ID, session_id=session.id, new_message=query
            ):
                # Check if the event contains valid content
                if event.content and event.content.parts:
                    # Filter out empty or "None" responses before printing
                    if (
                        event.content.parts[0].text != "None"
                        and event.content.parts[0].text
                    ):
                        print(f"{MODEL_NAME} > ", event.content.parts[0].text)
    else:
        print("No queries!")


print("✅ Helper functions defined.")

In [ ]:
root_agent = Agent(
    name="helpful_assistant",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description="A simple agent that can answer general questions.",
    instruction="You are a helpful assistant. Use Google Search for current info or if unsure.",
    tools=[google_search],
)

print("✅ Root Agent defined.")

In [ ]:
# runner = InMemoryRunner(agent=root_agent)
# print ("Runner created")

In [ ]:
# response = await runner.run_debug(
#     "What is Agent Development Kit from Google? What languages is the SDK available in?"
# )

In [ ]:
# resp = await runner.run_debug(" Can you find Black Friday deals on Iphone?")

In [ ]:
APP_NAME = "ChatBot"  # Application
USER_ID = user.email  # User
SESSION = user.sessionId  # Session

MODEL_NAME = "gemini-2.5-flash-lite"

print(SESSION)

In [ ]:

# Step 1: Create the LLM Agent
root_agent = Agent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="text_chat_bot",
    description="A text chatbot",  # Description of the agent's purpose
)

# Step 2: Set up Session Management
# InMemorySessionService stores conversations in RAM (temporary)
session_service = InMemorySessionService()

# Step 3: Create the Runner
runner = Runner(agent=root_agent, app_name=APP_NAME, session_service=session_service)

print("✅ Stateful agent initialized!")
print(f"   - Application: {APP_NAME}")
print(f"   - User: {USER_ID}")
print(f"   - Using: {session_service.__class__.__name__}")

In [ ]:
# Step 1: Create the same agent (notice we use LlmAgent this time)
chatbot_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="text_chat_bot",
    description="A text chatbot with persistent memory",
)

# Step 2: Switch to DatabaseSessionService
# SQLite database will be created automatically
db_url = "sqlite:///userslog.db"  # Local SQLite file
session_service = DatabaseSessionService(db_url=db_url)

# Step 3: Create a new runner with persistent storage
runner = Runner(agent=chatbot_agent, app_name=APP_NAME, session_service=session_service)

print("✅ Upgraded to persistent sessions!")
print(f"   - Database: my_agent_data.db")
print(f"   - Sessions will survive restarts!")

In [ ]:
# Re-define our app with Events Compaction enabled
research_app_compacting = App(
    name="research_app_compacting",
    root_agent=chatbot_agent,
    # This is the new part!
    events_compaction_config=EventsCompactionConfig(
        compaction_interval=3,  # Trigger compaction every 3 invocations
        overlap_size=1,  # Keep 1 previous turn for context
    ),
)

db_url = "sqlite:///my_agent_data.db"  # Local SQLite file
session_service = DatabaseSessionService(db_url=db_url)

# Create a new runner for our upgraded app
research_runner_compacting = Runner(
    app=research_app_compacting, session_service=session_service
)


print("✅ Research App upgraded with Events Compaction!")

In [ ]:
import sqlite3

def check_data_in_db():
    with sqlite3.connect("my_agent_data.db") as connection:
        cursor = connection.cursor()
        result = cursor.execute(
            "select app_name, session_id, author, content from events"
        )
        print([_[0] for _ in result.description])
        for each in result.fetchall():
            print(each)


check_data_in_db()

In [ ]:
 #Turn 1
await run_session(
    research_runner_compacting,
    "What is the latest news about AI in healthcare?",
    "compaction_demo",
)

# Turn 2
await run_session(
    research_runner_compacting,
    "Are there any new developments in drug discovery?",
    "compaction_demo",
)

# Turn 3 - Compaction should trigger after this turn!
await run_session(
    research_runner_compacting,
    "Tell me more about the second development you found.",
    "compaction_demo",
)

# Turn 4
await run_session(
    research_runner_compacting,
    "Who are the main companies involved in that?",
    "compaction_demo",
)

In [ ]:
async def auto_save_to_memory(callback_context):
    """Automatically save session to memory after each agent turn."""
    await callback_context._invocation_context.memory_service.add_session_to_memory(
        callback_context._invocation_context.session
    )


print("✅ Callback created.")

In [ ]:
import requests

def getProductInfo(productName: str) -> dict:
    """
      This method return all matches products from API
    """
    url= f'https://dummyjson.com/products/search?q={productName}'
    print(url)
    resp = requests.get(url, timeout=10)

    # Raise if HTTP status is not 2xx
    resp.raise_for_status()
    products= []
    # print(products, len(products))
    # if len(products) == 0:
    #     return {
    #         'id': 0
    #         ,'title': f"Sorry, I don't have information for {productName}. Available products: 0"
    #     }
    data=resp.json()
    counter=0
    count=0
    # print(type(data), len(data) )
    for item in data['products']:
        if productName.lower() in item.get('title').lower():
            products.append(item)
            count += 1
        counter+= 1
    if len(products) == 0:
        return {
            'id': 0
            ,'title': f"Sorry, I don't have information for {productName}. Available products: 0"
        }
    # print(counter, count)
    # Parse and return JSON body
    return products
    


print(getProductInfo('Iphone 6'))

In [ ]:
print(user.sessionId)